In [1]:
# Add ./python/ to path
import sys
sys.path.append('python/')

from data_prep import *
from charts import *
from players import *
from video_analysis import *
from team_sheets import *

# # Load data
# game_df = team_sheets()
# players_df = players(game_df)
# players_agg_df = players_agg(players_df)
# lineouts_df = lineouts()
# set_piece_df = set_piece_results()
# analysis = game_stats()
# players_table = players_table_data(players_df, players_agg_df)
 
# # Save data
# game_df.to_csv('data/game.csv', index=False)
# players_df.to_csv('data/players.csv', index=False)
# players_agg_df.to_csv('data/players_agg.csv', index=False)
# lineouts_df.to_csv('data/lineouts.csv', index=False)
# set_piece_df.to_csv('data/set_piece.csv', index=False)
# analysis.to_csv('data/analysis.csv', index=False)
# update_season_summaries(game_df, seasons=["2024/25"])


game_df = pd.read_json('data/game.json')
players_df = pd.read_json('data/players.json')
players_agg_df = pd.read_json('data/players_agg.json')
lineouts_df = pd.read_json('data/lineouts.json')
set_piece_df = pd.read_json('data/set_piece.json')
analysis = pd.read_json('data/analysis.json')
players_table = pd.read_json('data/player_table.json')

# # One-off charts (only source data needs updating)
# captains_chart(file='Charts/captains.html')
# results_chart(file='Charts/results.html')
# plot_games_by_player(file='Charts/appearances.html')
# plot_starts_by_position(file='Charts/positions.html')
# card_chart(file='Charts/cards.html')
# points_scorers_chart(file='Charts/points.html')
# team_sheets_chart(file='Charts/team-sheets.html')
# set_piece_h2h_chart(file='Charts/set-piece.html')
# squad_continuity_chart(file='Charts/continuity.html')

# # Self-contained charts (chart needs updating)
# game_stats_charts(analysis, file='Charts/video_analysis.html')
# lineout_success(types=types, file='Charts/lineouts.html')

In [3]:
players_df

,Date,GameID,Squad,Season,Competition,GameType,Opposition,Home/Away,PF,PA,Result,Captain,VC1,VC2,Number,Player,Position,Position_specific,PositionType
0,2021-05-22,2021-05-22_1_Crawley,1st,2021/22,Friendly,Friendly,Crawley,H,34,12,W,Jack Andrews,James Funnell,None,1,James Funnell,Prop,Prop,Forwards
1,2021-06-12,2021-06-12_1_University of Brighton,1st,2021/22,Friendly,Friendly,University of Brighton,A,0,43,L,Jack Andrews,James Funnell,None,1,James Funnell,Prop,Prop,Forwards
2,2021-07-03,2021-07-03_1_Metropolitan Police,1st,2021/22,Friendly,Friendly,Metropolitan Police,H,29,28,W,Jack Andrews,None,None,1,James Funnell,Prop,Prop,Forwards
3,2021-08-28,2021-08-28_1_London Irish,1st,2021/22,Friendly,Friendly,London Irish,A,10,33,L,Jack Andrews,Sam Lindsay-McCall,None,1,Sean Morgan,Prop,Prop,Forwards
4,2021-09-04,2021-09-04_1_Horsham,1st,2021/22,Friendly,Friendly,Horsham,A,26,56,L,Jack Andrews,James Funnell,None,1,James Funnell,Prop,Prop,Forwards
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3070,2021-07-03,2021-07-03_1_Metropolitan Police,1st,2021/22,Friendly,Friendly,Metropolitan Police,H,29,28,W,Jack Andrews,None,None,25,Henry Weller,None,Bench,Bench
3071,2023-08-19,2023-08-19_1_Old Colfeians,1st,2023/24,Friendly,Friendly,Old Colfeians,H,19,36,L,Connor Leggat,Oli Billin,Sam Lindsay-McCall,25,Ben Beard,None,Bench,Bench
3072,2023-09-02,2023-09-02_1_Heathfield & Waldron,1st,2023/24,Friendly,Friendly,Heathfield & Waldron,H,0,5,L,Max Crawley-Moore,None,None,25,Colm Geraghty,None,Bench,Bench
3073,2024-02-13,2024-02-13_2_Crowborough,2nd,2023/24,Counties 4 Sussex,League,Crowborough,H,10,7,W,Ben Watkinson,Max Crawley-Moore,None,25,Jack Andrews,None,Bench,Bench


In [2]:
def plot_starts_by_position(df=None, file=None):

    facet_selection = alt.param(
        bind=alt.binding_radio(options=["PositionType", "Position", "Position_specific"], name='Facet by'),
        value="Position_specific"
    )

    season_selection = alt.param(
        bind=alt.binding_radio(options=["All", *seasons[::-1]], name="Season"), 
        value="2024/25" 
    )
    squad_selection = alt.param(
        bind=alt.binding_radio(options=["1st", "2nd", "Total"], name="Squad"),
        value="Total"
    )

    position_order = [
        "Forwards", 
        "Prop", 
        "Hooker", 
        "Second Row" 
        "Back Row", "Flanker", "Number 8",
        "Backs",
        "Scrum Half", 
        "Fly Half", 
        "Centre", 
        "Back Three", "Wing", "Full Back" 
    ]

    chart = (
        alt.Chart(df if df is not None else {"name": "df", "url":'https://raw.githubusercontent.com/samnlindsay/egrfc-stats/main/data/players.json',"format":{'type':"json"}})
        .mark_bar(color="#202947")
        .encode(
            x=alt.X('count()', axis=alt.Axis(title=None, orient="top")),
            y=alt.Y('Player:N', sort='-x', title=None),
            facet=alt.Facet(
                'facet_by:N',
                columns=1,
                header=alt.Header(title=None, labelFontSize=20, labelOrient="bottom", labelAngle=0, labelPadding=-5, labelColor="#20294780", labelAnchor="end", labelAlign="right"), 
                spacing=0, 
                sort=position_order,
                align="each"
            ),
            tooltip=[
                "Player:N", 
                alt.Tooltip("facet_by:N", title="Position"),
                alt.Tooltip("count()", title="Starts"), 
                'Rank:Q',
                'N:Q',
                'SeasonApps:Q',
                'SeasonStarts:Q',
                'tiebreaker:Q',
            ],      
            order=alt.Order('count()', sort='descending'),
            opacity=alt.condition(
                '(datum.facet_by == "Back Row" || datum.facet_by == "Back Three") && (datum.Rank <= 3)',
                alt.value(1),
                alt.value(0.5)
            )
        )
        .resolve_scale(y="independent", x="shared")
        .transform_joinaggregate(
            SeasonApps="count()",
            groupby=["Player", "Squad", "Season"]
        )
        .transform_filter("datum.Number <= 15")
        .properties(width=150, height=alt.Step(14), title=alt.Title(text="Starts by Position", subtitle="Not including bench appearances."))
        .add_params(season_selection, squad_selection, facet_selection)
        .transform_calculate(
            # Dynamically assign the value of the selected encoding
            facet_by=f"datum[{facet_selection.name}]"
        )
        .transform_joinaggregate(TotalGames="count()", groupby=["Player", "facet_by"])
        .transform_filter(f"datum.Season == {season_selection.name} | {season_selection.name} == 'All'")
        .transform_filter(f"datum.Squad == {squad_selection.name} | {squad_selection.name} == 'Total'")
        .transform_joinaggregate(Games="count()", groupby=["Player", "facet_by"])
        .transform_joinaggregate(SeasonStarts="count()", groupby=["Player", "Squad", "Season"])
        .transform_calculate(
            tiebreaker="datum.Games + 0.1*datum.SeasonStarts"
        )
        .transform_window(
            Rank="dense_rank(tiebreaker)",
            N="rank(tiebreaker)",
            groupby=["facet_by"],
            sort=[{"field": "tiebreaker", "order": "descending"}]
        )
        .transform_filter(f"datum.Rank <= 5")
    )
    if file:
        chart.save(file, embed_options={'renderer':'svg', 'actions': {'export': True, 'source':False, 'editor':True, 'compiled':False} })
        hack_params_css(file)

    return chart  

plot_starts_by_position(players_df)

alt.Chart(...)

In [3]:
players_agg_df[players_agg_df["Season"] == "2024/25"].groupby("Player")["Tries"].sum().sort_values(ascending=False).head(10)

Player
Will Roberts        40.0
Noah Roberts        35.0
Ted Hardisty        30.0
Jack Billin         30.0
Jake Radcliffe      30.0
Ryan Morlen         25.0
Ali Moffatt         25.0
Sam Spriddell       20.0
Paddy Shepherd      20.0
Chris May-Miller    15.0
Name: Tries, dtype: float64

In [3]:
lineouts_df

,Squad,Season,Opposition,Numbers,Call,CallType,Setup,Movement,Area,Drive,Crusaders,Transfer,Flyby,Hooker,Jumper,Won
0,1st,2021/22,London Irish,5,,Other,None,Jump,Middle,False,False,False,NaN,Ben,Sam,1
1,1st,2021/22,London Irish,4,Snap,4-man only,None,Jump,Front,False,False,False,NaN,Ben,Gus,1
2,1st,2021/22,London Irish,6,Even,Old,None,Jump,Middle,True,False,False,NaN,Ben,Sam,1
3,1st,2021/22,London Irish,6,,Other,None,Jump,Middle,True,False,False,NaN,Ben,Sam,1
4,1st,2021/22,London Irish,6,Green,Old,None,Jump,Back,False,False,False,NaN,Ben,Rory,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,2nd,2024/25,Horsham,7,H3,New,H,Jump,Middle,False,False,False,NaN,George,Paddy,1
1022,2nd,2024/25,Horsham,7,H3,New,H,Jump,Middle,False,False,False,NaN,George,,1
1023,2nd,2024/25,Horsham,7,H3,New,H,Jump,Middle,False,False,False,NaN,George,Sammy,1
1024,2nd,2024/25,Horsham,7,H1,New,H,Jump,Back,False,False,False,NaN,George,Sammy,1


In [3]:
plot_games_by_player(df=players_df, file='Charts/appearances.html')

Updated Charts/appearances.html


alt.Chart(...)

In [3]:
df = lineouts_df.copy()
df["Call"] = df["Call"].apply(lambda x: x[-1] if len(x) > 0 and x[-1] in ["1", "2", "3", "*"] else None)
df
(
    alt.Chart(df)
    .mark_rect()
    .encode(
        x=alt.X('Call:O', title='Lineout'),
        y=alt.Y('Setup:O', title='Team'),
        color=alt.Color(
            'Won:Q', 
            title='Success', 
            aggregate='mean', 
            scale=alt.Scale(domain=[0.5, 1], scheme="redyellowgreen"),
        ),
        tooltip=[
            alt.Tooltip('Call:O', title='Call'),
            alt.Tooltip('Setup:O', title='Team'),
            alt.Tooltip('Won:O', aggregate='mean', title='Success', format='.0%'),
            alt.Tooltip('Count:Q', title='Count', aggregate='count'),
        ]
    )
    .transform_filter(alt.datum.Call != None)
    .transform_filter(alt.datum.Season == "2024/25")
    .transform_filter(alt.datum.Numbers == 7)
    .transform_filter(alt.datum.Squad == "1st")
    .properties(
        title="Lineout Success",
        width=alt.Step(100),
        height=alt.Step(100),
    )
)

alt.Chart(...)

In [101]:
import pandas as pd
import numpy as np

def play(row):
    if row["Drive"]:
        return "Cold"
    elif row["Flyby"]:
        return "Flyby"
    elif row["Crusaders"]:
        return "Crusaders"
    elif row["Call"] == "Orange":
        return "Orange"
    elif row["Won"] == 1:
        return "Hot"
    else:
        return None


df["Play"] = df.replace(np.nan, None).apply(play, axis=1)
df = df.dropna(subset=["Play"])

play_df = df[df["Squad"]=="1st"].groupby(["Season", "Numbers", "Play"]).agg({"Won": "count"}).reset_index().rename(columns={"Won": "Count"}).sort_values(["Season", "Numbers", "Play"])

alt.Chart(play_df).mark_bar().encode(
    x=alt.X("Numbers:O", title="Numbers"),
    y=alt.Y("Percentage:Q", title="Proportion", stack="normalize", axis=alt.Axis(format=".0%")),
    color=alt.Color(
        "Play:O", 
        title="Play", 
        scale=alt.Scale(
            domain=["Cold", "Flyby", "Crusaders", "Hot"], 
            range=["#00a", "#33c", "#66f", "#c30"]
        )
    ),
    column=alt.Column("Season:O", title=None),
    tooltip=[
        alt.Tooltip("Play:O", title="Play"),
        alt.Tooltip("Count:Q", title="Count"),
    ]
).transform_joinaggregate(
    SeasonTotal="sum(Count)",
    groupby=["Season"]
).transform_calculate(
    Percentage="datum.Count / datum.SeasonTotal"
).properties(
    title=alt.Title(
        text="Lineout Plays",
        subtitle="How the ball is played away from successful lineouts. 'Hot' also includes any time the ball is won but not cleanly regathered.",
    ),
    width=alt.Step(40),
    height=300,
)

alt.Chart(...)

In [60]:
cont_df = team_sheets()

df = cont_df[(cont_df["Squad"] == "1st") & (cont_df["Season"] == "2024/25")].reset_index()


line = (
    alt.Chart(df)
    .mark_line(point=True, size=1)
    .transform_fold(
        ["Starters_retained", "Forwards_retained", "Backs_retained"],
        as_=["Retained", "Count"]
    )
    .transform_calculate(
        Retained="replace(datum.Retained, '_retained', '')"
    )
    .encode(
        x=alt.X("GameID:O", title="Game", axis=alt.Axis(labelOverlap=True), sort=alt.EncodingSortField(field="index")),
        y=alt.Y("Count:Q", title="Players Retained", scale=alt.Scale(domain=[0, 15])),
        color=alt.Color(
            "Retained:N", 
            title=None, 
            scale=alt.Scale(
                domain=["Starters", "Forwards", "Backs"], 
                range=["black", "#202946", "#981515"]
            ),
            legend=alt.Legend(orient="bottom-left", direction="horizontal")
        ),
        tooltip=[
            alt.Tooltip("GameID:O", title="Game"),
            alt.Tooltip("Retained:N", title="Players"),
            alt.Tooltip("Count:Q", title="Retained from previous game"),
        ],
    )
)

# Add horizontal line at 15
line = line + (
    alt.Chart(pd.DataFrame({"y": [15, 8, 7], "Retained": ["Starters", "Forwards", "Backs"]}))
    .mark_rule(color="red", strokeWidth=4, opacity=0.2, strokeDash=[5, 5])
    .encode(
        y=alt.Y("y:Q", title="Players Retained", scale=alt.Scale(domain=[0, 15])),
        color=alt.Color(
            "Retained:N", 
            title=None, 
            scale=alt.Scale(
                domain=["Starters", "Forwards", "Backs"], 
                range=["black", "#202946", "#981515"]
            )
        ),
    )
)
line = line.properties(
    title=alt.Title(
        text="Squad Continuity",
        subtitle="Number of players retained from previous game. Forwards and backs are shown separately.",
    ),
    width=alt.Step(25),
    height=300,
)
line

alt.LayerChart(...)

In [30]:
import duckdb

df = team_sheets()

# Extract unique players
position_cols = [str(i) for i in range(1, 30)]
player_names = pd.unique(df[position_cols].values.ravel())
player_df = pd.DataFrame({'player_id': player_names}).dropna()

# Create DuckDB connection
con = duckdb.connect(database=':memory:')

# Create tables
con.execute("""
CREATE TABLE player (
    player_id TEXT PRIMARY KEY
);
""")

con.execute("""
CREATE TABLE game (
    game_id TEXT PRIMARY KEY,
    date DATE,
    season TEXT,
    competition TEXT,
    opposition TEXT,
    score TEXT,
    squad TEXT,
    home_away TEXT,
    game_type TEXT,
    pf INTEGER,
    pa INTEGER,
    result TEXT,
    forwards_retained INTEGER,
    backs_retained INTEGER,
    starters_retained INTEGER,
    fullsquad_retained INTEGER,
    captain TEXT,
    vc1 TEXT,
    vc2 TEXT
);
""")

con.execute("""
CREATE TABLE appearance (
    game_id TEXT,
    player_id TEXT,
    position_number INTEGER,
    unit TEXT,
    position TEXT,
    PRIMARY KEY (game_id, player_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (player_id) REFERENCES player(player_id)
);
""")

# Insert players and games
con.register("players_df", player_df)
con.execute("INSERT INTO player SELECT * FROM players_df")

game_cols = [
    'GameID', 'Date', 'Season', 'Competition', 'Opposition', 'Score',
    'Squad', 'Home/Away', 'GameType', 'PF', 'PA', 'Result',
    'Forwards_retained', 'Backs_retained', 'Starters_retained', 'FullSquad_retained',
    'Captain', 'VC1', 'VC2'
]
game_df = df[game_cols].copy()
game_df.columns = [
    'game_id', 'date', 'season', 'competition', 'opposition', 'score',
    'squad', 'home_away', 'game_type', 'pf', 'pa', 'result',
    'forwards_retained', 'backs_retained', 'starters_retained', 'fullsquad_retained',
    'captain', 'vc1', 'vc2'
]
con.register("games_df", game_df)
con.execute("INSERT INTO game SELECT * FROM games_df")

# Prepare and insert appearances
appearance_records = []
for _, row in df.iterrows():
    game_id = row['GameID']
    for pos in range(1, 30):
        player = row.get(str(pos))
        if pd.notna(player):
            appearance_records.append((game_id, player, pos))

appearance_df = pd.DataFrame(appearance_records, columns=['game_id', 'player_id', 'position_number'])
# Add unit and position columns
appearance_df['unit'] = appearance_df['position_number'].apply(lambda x: 'Forwards' if x <= 8 else 'Backs' if x <= 15 else 'Bench')
appearance_df['position'] = appearance_df['position_number'].map({
    1: 'Prop', 2: 'Hooker', 3: 'Prop', 4: 'Second Row', 5: 'Second Row',
    6: 'Flanker', 7: 'Flanker', 8: 'Number 8', 9: 'Scrum Half', 10: 'Fly Half',
    11: 'Wing', 12: 'Centre', 13: 'Centre', 14: 'Wing', 15: 'Full Back'
}).fillna('Bench')

con.register("appearance_df", appearance_df)
con.execute("INSERT INTO appearance SELECT * FROM appearance_df")

# Show preview from each table
# players_preview = con.execute("SELECT * FROM player").fetchdf()
# games_preview = con.execute("SELECT * FROM game").fetchdf()
# appearances_preview = con.execute("SELECT * FROM appearance").fetchdf()

# players_preview, games_preview, appearances_preview.head(10)


In [ ]:
import altair as alt

# Query: Number of times each player appeared in a given position
query = """
SELECT a.player_id, a.position, a.unit, g.squad, g.season, COUNT(*) as appearances
FROM appearance a
JOIN game g 
    ON a.game_id = g.game_id
GROUP BY a.player_id, a.position, a.unit, g.squad, g.season
ORDER BY position, appearances DESC
"""`

pos_df = con.execute(query).fetchdf()

pos_df

,player_id,position,unit,squad,season,appearances
0,Dave Bridges,Bench,Bench,1st,2023/24,12
1,Ryan Morlen,Bench,Bench,1st,2021/22,11
2,Ben Watkinson,Bench,Bench,2nd,2024/25,9
3,Oli Billin,Bench,Bench,1st,2021/22,8
4,Evan Gosling,Bench,Bench,2nd,2024/25,8
...,...,...,...,...,...,...
1001,Titch Mitchell,Wing,Backs,1st,2022/23,1
1002,Sean Madden,Wing,Backs,2nd,2021/22,1
1003,Andy Blackmore,Wing,Backs,2nd,2021/22,1
1004,Oscar Morgan,Wing,Backs,2nd,2023/24,1


In [29]:
season_selection = alt.param(
    bind=alt.binding_radio(options=["All", *seasons[::-1]], name="season"), 
    value=max(seasons) 
)
squad_selection = alt.param(
    bind=alt.binding_radio(options=["1st", "2nd", "Total"], name="squad"),
    value="Total"
)
min_selection = alt.param(
    bind=alt.binding_range(name="Minimum Starts", min=1, max=20, step=1),
    value=min
)

position_order = [
    "Prop", "Hooker", "Second Row", "Flanker", "Number 8", 
    "Scrum Half", "Fly Half", "Centre", "Wing", "Full Back",
    "Bench"
]

# Altair chart: heatmap of appearances by player and position
chart = (
    alt.Chart(pos_df).mark_bar()
        .encode(
            x=alt.X('sum(appearances)', axis=alt.Axis(title=None, orient="top")),
            y=alt.Y('player_id:N', sort='-x', title=None),
            facet=alt.Facet(
                "position:O", 
                columns=5,  
                header=alt.Header(title=None, labelFontSize=36, labelOrient="top"), 
                spacing=0, 
                sort=position_order,
                align="each"
            ),
            tooltip=[
                "player_id:N", 
                "position:N", 
                alt.Tooltip("sum(appearances)", title="Starts"), 
                "TotalGames:Q",
            ],      
        )
        .resolve_scale(y="independent", x="independent")
        .properties(width=150, height=alt.Step(14), title=alt.Title(text="Starts by Position"))
        .add_params(season_selection, squad_selection)
        .transform_joinaggregate(TotalGames="sum(appearances)", groupby=["player_id", "position"])
        # .transform_window(
        #     Rank="rank(count())",
        #     groupby=["Position"],
        #     sort=[alt.SortField("count()", order="descending")]
        # )
        # .transform_filter(f"datum.TotalGames >= {min_selection.name}")
        .transform_filter(f"datum.season == {season_selection.name} | {season_selection.name} == 'All'")
        .transform_filter(f"datum.squad == {squad_selection.name} | {squad_selection.name} == 'Total'")
        # .transform_joinaggregate(Games="count()", groupby=["Player", "Position"])
        # .transform_window(
        #     Rank="dense_rank(Games)",
        #     groupby=["Position"],
        #     sort=[{"field": "Games", "order": "descending"}]
        # )
)

chart

alt.Chart(...)